In [10]:
import subprocess
import glob
import os
from os.path import exists
import re
#from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Pool

In [11]:
fastq_dir = '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/01.RawData/'

In [12]:
list_of_fq = {}
for (dirpath, dirnames, filenames) in os.walk(fastq_dir):
    for filename in filenames:
        if filename.endswith('fq.gz'):
            list_of_fq[filename] = os.sep.join([dirpath, filename])

list_of_fq

{'DM2_FKDL230359836-1A_22CNLCLT3_L4_1.fq.gz': '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/01.RawData/DM2/DM2_FKDL230359836-1A_22CNLCLT3_L4_1.fq.gz',
 'DM2_FKDL230359836-1A_22CNLCLT3_L4_2.fq.gz': '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/01.RawData/DM2/DM2_FKDL230359836-1A_22CNLCLT3_L4_2.fq.gz',
 'DM1_FKDL230359835-1A_22CNLCLT3_L4_2.fq.gz': '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/01.RawData/DM1/DM1_FKDL230359835-1A_22CNLCLT3_L4_2.fq.gz',
 'DM1_FKDL230359835-1A_22CNLCLT3_L4_1.fq.gz': '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/01.RawData/DM1/DM1_FKDL230359835-1A_22CNLCLT3_L4_1.fq.gz',
 'DM3_FKDL230359837-1A_22CNLCLT3_L4_2.fq.gz': '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/01.RawData/DM3/DM3_FKDL230359837-1A_22CNLCLT3_L4_2.fq.gz',
 'DM3_FKDL230359837-1A_22CNLCLT3_L4_1.fq.gz': '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/01.RawData/DM3/

In [13]:
import re

list_of_sample = set([re.sub('_\d\.fq\.gz$', '', fq) for fq in list_of_fq.keys()])

print(list_of_sample)  # This will print something like: {'DM2', 'DM1', 'C12C', ...}
print(len(list_of_sample))  # This will print the number of unique samples


{'DM3_FKDL230359837-1A_22CNLCLT3_L4', 'C12A_FKDL230359832-1A_22CT5CLT3_L2', 'C12C_FKDL230359834-1A_22CNLCLT3_L8', 'C12B_FKDL230359833-1A_22CNLCLT3_L8', 'DM2_FKDL230359836-1A_22CNLCLT3_L4', 'DM1_FKDL230359835-1A_22CNLCLT3_L4'}
6


In [14]:
QC_output_dir = '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/QC_fastp/'
# Ensure the output directory exists
os.makedirs(QC_output_dir, exist_ok=True)

In [26]:
def run_fastp(sample):
    r1_input = list_of_fq[f'{sample}_1.fq.gz']
    r2_input = list_of_fq[f'{sample}_2.fq.gz']
    r1_output = os.path.join(QC_output_dir, f'{sample}_1_clean.fq.gz')
    r2_output = os.path.join(QC_output_dir, f'{sample}_2_clean.fq.gz')
    html_report = os.path.join(QC_output_dir, f'{sample}_report.html')
    
    command = [
        'fastp',
        '-i', r1_input,
        '-I', r2_input,
        '-o', r1_output,
        '-O', r2_output,
        '-h', html_report
    ]
    
    subprocess.run(command, check=True)

# Convert list_of_sample from a set to a list, as Pool.map requires a list
list_of_sample = list(list_of_sample)

# Use a pool to run fastp on multiple samples in parallel
with Pool(3) as p:
    p.map(run_fastp, list_of_sample)


Read1 before filtering:
total reads: 80663152
total bases: 12099472800
Q20 bases: 11457706049(94.6959%)
Q30 bases: 10856820785(89.7297%)

Read2 before filtering:
total reads: 80663152
total bases: 12099472800
Q20 bases: 11876305922(98.1556%)
Q30 bases: 11658547351(96.3558%)

Read1 after filtering:
total reads: 80131180
total bases: 9077033498
Q20 bases: 8935716599(98.4431%)
Q30 bases: 8828769181(97.2649%)

Read2 after filtering:
total reads: 80131180
total bases: 9077033498
Q20 bases: 8947445343(98.5724%)
Q30 bases: 8822127515(97.1917%)

Filtering result:
reads passed filter: 160262360
reads failed due to low quality: 1001670
reads failed due to too many N: 62274
reads failed due to too short: 0
reads with adapter trimmed: 100474336
bases trimmed due to adapters: 5893663962

Duplication rate: 9.45277%

Insert size peak (evaluated by paired-end reads): 97

JSON report: fastp.json
HTML report: /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/QC_fastp/DM1_FKDL230359835-

#### tools like Bowtie2 or BWA are more commonly used for ATAC-seq data as they are optimized for speed and efficiency in aligning non-spliced reads.

#### hisat2 might introduce gap in alignments

In [15]:
output_dir = '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
sam_path = output_dir + '/1_sam/'
bam_path = output_dir + '/2_bam/'
if not os.path.exists(sam_path):
    os.makedirs(sam_path)
    
if not os.path.exists(bam_path):
    os.makedirs(bam_path)

In [16]:
! /home/hu/Downloads/bowtie2-2.5.1-linux-x86_64/bowtie2-build /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa /media/HDD2/Genomes/Ath_Ensembl56/Ath_bowtie2_index

Settings:
  Output files: "/media/HDD2/Genomes/Ath_Ensembl56/Ath_bowtie2_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 29870385
Using parameters --bmax 22402789 --dcv 1024
  Doing ahead-of-time memory usag

In [30]:
def run_bowtie2(sample):
    r1_input = os.path.join(QC_output_dir, f'{sample}_1_clean.fq.gz')
    r2_input = os.path.join(QC_output_dir, f'{sample}_2_clean.fq.gz')
    # now you want to split it and take just the first part ('DM2')
    simplified_sample = re.split('_', sample)[0]
    sam_output = os.path.join(sam_path, f'{simplified_sample}.sam')
    
    command = [
        '/home/hu/Downloads/bowtie2-2.5.1-linux-x86_64/bowtie2',
        '-x', '/media/HDD2/Genomes/Ath_Ensembl56/Ath_bowtie2_index',  # replace with your actual reference path
        '-1', r1_input,
        '-2', r2_input,
        '-S', sam_output,
        '-p', '31' # threads
    ]
    
    subprocess.run(command, check=True)

# Run Bowtie2 alignment on all samples in parallel
with Pool(4) as p:
    p.map(run_bowtie2, list_of_sample)


80131180 reads; of these:
  80131180 (100.00%) were paired; of these:
    1626271 (2.03%) aligned concordantly 0 times
    50650802 (63.21%) aligned concordantly exactly 1 time
    27854107 (34.76%) aligned concordantly >1 times
    ----
    1626271 pairs aligned concordantly 0 times; of these:
      582436 (35.81%) aligned discordantly 1 time
    ----
    1043835 pairs aligned 0 times concordantly or discordantly; of these:
      2087670 mates make up the pairs; of these:
        1404524 (67.28%) aligned 0 times
        204914 (9.82%) aligned exactly 1 time
        478232 (22.91%) aligned >1 times
99.12% overall alignment rate
87521233 reads; of these:
  87521233 (100.00%) were paired; of these:
    1545221 (1.77%) aligned concordantly 0 times
    52838444 (60.37%) aligned concordantly exactly 1 time
    33137568 (37.86%) aligned concordantly >1 times
    ----
    1545221 pairs aligned concordantly 0 times; of these:
      526862 (34.10%) aligned discordantly 1 time
    ----
    10183

In [16]:
glob.glob(os.path.join(sam_path, '*.sam'))

[]

In [17]:
sam_files = glob.glob(os.path.join(sam_path, '*.sam'))

# Extracting the sample names from the file paths
list_of_sample = [os.path.basename(sam_file).replace('.sam', '') for sam_file in sam_files]


In [18]:
list_of_sample

[]

In [35]:
def process_sam(sample):
    sam_input = os.path.join(sam_path, f'{sample}.sam')
    bam_output = os.path.join(bam_path, f'{sample}_sorted.bam')
    
    # Convert SAM to BAM
    command1 = [
        'samtools',
        'view',
        '-@', '30',  # using multi threads
        '-bS', sam_input
    ]
    
    # Sort BAM
    command2 = [
        'samtools',
        'sort',
        '-@', '30',  # using multi threads
        '-o', bam_output
    ]
    
    # Index BAM
    command3 = [
        'samtools',
        'index',
        '-@', '30',  # using multi threads
        bam_output
    ]
    
    # Run commands
    with subprocess.Popen(command1, stdout=subprocess.PIPE) as p1:
        with subprocess.Popen(command2, stdin=p1.stdout) as p2:
            p2.wait()
    subprocess.run(command3, check=True)
    
    
# Process SAM files for all samples in parallel
with Pool(4) as p:
    p.map(process_sam, list_of_sample)


[bam_sort_core] merging from 60 files and 30 in-memory blocks...
[bam_sort_core] merging from 60 files and 30 in-memory blocks...
[bam_sort_core] merging from 60 files and 30 in-memory blocks...
[bam_sort_core] merging from 90 files and 30 in-memory blocks...
[bam_sort_core] merging from 60 files and 30 in-memory blocks...
[bam_sort_core] merging from 60 files and 30 in-memory blocks...


In [19]:
# Setting up directories for cleaned BAM files and merged BAM file
clean_bam_path = os.path.join(output_dir, '3_clean_bam')
merged_bam_path = os.path.join(output_dir, '4_merged_bam')

os.makedirs(clean_bam_path, exist_ok=True)
os.makedirs(merged_bam_path, exist_ok=True)


In [25]:
bam_path

'/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/2_bam/'

In [26]:
bam_files = glob.glob(os.path.join(bam_path, '*.bam'))

# Extracting the sample names from the file paths
list_of_sample = [os.path.basename(bam_file).replace('.bam', '') for bam_file in bam_files]


In [29]:
list_of_sample = [sample.replace('_sorted', '') for sample in list_of_sample]
list_of_sample

['DM3', 'DM1', 'C12C', 'DM2', 'C12A', 'C12B']

In [30]:
import os
import subprocess
from multiprocessing import Pool

def filter_bam(sample):
    bam_input = os.path.join(bam_path, f'{sample}_sorted.bam')
    
    bam_output_temp1 = os.path.join(clean_bam_path, f'{sample}_temp1.bam')
    bam_output_temp2 = os.path.join(clean_bam_path, f'{sample}_temp2.bam')
    bam_output_final = os.path.join(clean_bam_path, f'{sample}_clean.bam')

    # Filter using Sambamba
    command1 = [
        'sambamba',
        'view',
        '-f', 'bam',
        '-F', 'not (unmapped or mate_is_unmapped or secondary_alignment or failed_quality_control)',
        '--nthreads', '20',
        '-o', bam_output_temp1,
        bam_input
    ]

    # Mark duplicates using Sambamba
    command2 = [
        'sambamba',
        'markdup',
        '--nthreads', '20',
        bam_output_temp1,
        bam_output_temp2
    ]

    # Filter low-quality reads using Samtools
    command3 = [
        'samtools',
        'view',
        '-q', '20',
        '-b',
        bam_output_temp2,
        '-@', '25',
        '-o', bam_output_final
    ]

    # Indexing the final bam file using Samtools
    command4 = [
        # 'samtools',
        'sambamba',
        'index',
        '-@', '20',
        bam_output_final
    ]

    # Execute the commands
    subprocess.run(command1, check=True)
    subprocess.run(command2, check=True)
    subprocess.run(command3, check=True)
    subprocess.run(command4, check=True)

    # Clean up temporary files
    os.remove(bam_output_temp1)
    os.remove(bam_output_temp2)

    # Also remove the associated .bai index files if they exist
    for temp_file in [bam_output_temp1, bam_output_temp2]:
        bai_file = temp_file + ".bai"
        if os.path.exists(bai_file):
            os.remove(bai_file)

# Filter BAM files for all samples in parallel
with Pool(4) as p:
    p.map(filter_bam, list_of_sample)


finding positions of the duplicate reads in the file...
finding positions of the duplicate reads in the file...
finding positions of the duplicate reads in the file...
finding positions of the duplicate reads in the file...
  sorted 79309995 end pairs
     and 0 single ends (among them 0 unmatched pairs)
  collecting indices of duplicate reads...   sorted 82244886 end pairs
     and 0 single ends (among them 0 unmatched pairs)
  collecting indices of duplicate reads...   done in 7338 ms
  found 42538752 duplicates
collected list of positions in 1 min 33 sec
marking duplicates...
  done in 6896 ms
  found 42807056 duplicates
collected list of positions in 1 min 34 sec
marking duplicates...
  sorted 86732844 end pairs
     and 0 single ends (among them 0 unmatched pairs)
  collecting indices of duplicate reads...   done in 8599 ms
  found 75319478 duplicates
collected list of positions in 2 min 4 sec
marking duplicates...
total time elapsed: 2 min 48 sec
  sorted 92664430 end pairs
     

In [36]:
! samtools idxstats /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/3_clean_bam/DM1_clean.bam

1	30427671	28389619	0
2	19698289	18428894	0
3	23459830	22719066	0
4	18585056	17390074	0
5	26975502	25169344	0
Mt	366924	1344801	0
Pt	154478	4735664	0
*	0	0	0


##### keep_regions.bed as
"""1	0	30427671
2	0	19698289
3	0	23459830
4	0	18585056
5	0	26975502
""" are created

In [37]:
clean_bam_path

'/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/3_clean_bam'

In [38]:
bed_file = '/media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/3_clean_bam/keep_regions.bed'

In [39]:
intersect_bam_path = os.path.join(output_dir, '4_intersect_bam')
os.makedirs(intersect_bam_path, exist_ok=True)

In [45]:
def intersect_bam(sample):
    bam_input = os.path.join(clean_bam_path, f'{sample}_clean.bam')
    output_file = os.path.join(intersect_bam_path, f'{sample}_clean_filtered.bam')

    command = [
        'bedtools',
        'intersect',
        '-a', bam_input,
        '-b', bed_file,
        '-wa'
    ]
    
    # Capture standard output and error
    with open(output_file, 'wb') as out_file:
        result = subprocess.run(command, stdout=out_file, stderr=subprocess.PIPE)
    
    # If there's an error, print the error message
    if result.returncode != 0:
        print(f"Error processing {sample}:\n{result.stderr.decode('utf-8')}")
        return  # Exit the function or handle the error as needed

# Filter BAM files for all samples in parallel
with Pool(4) as p:
    p.map(intersect_bam, list_of_sample)


In [46]:
def index_bam(sample):
    bam_file_to_index = os.path.join(intersect_bam_path, f'{sample}_clean_filtered.bam')
    command = [
        'sambamba',
        'index',
        '-t', '20',  # Number of threads
        bam_file_to_index
    ]
    
    # Execute the command
    result = subprocess.run(command, capture_output=True, text=True)

    # If there's an error, print the error message
    if result.returncode != 0:
        print(f"Error indexing {sample}:\n{result.stderr}")
        return  # Exit the function or handle the error as needed

# Index BAM files for all samples in parallel
with Pool(4) as p:
    p.map(index_bam, list_of_sample)
